In [18]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
pip install git+https://github.com/csebuetnlp/normalizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-x870_tn7
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-x870_tn7
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py) ... done


In [20]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch

model = AutoModelForTokenClassification.from_pretrained("csebuetnlp/banglabert", num_labels=13) # num_labels will be the total no differents of NER tags ...
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert", use_fast=True,)

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.weight', 'classifier.bias']
You shoul

In [21]:
model.config.architectures = ['ElectraForTokenClassification']

In [22]:
fp=open("train.txt", "r").read().split("\n\n") #train data
total_sentence=len(fp)
# fn=open("newtrain.txt","a")
# count=1
x=[]
y=[]
d=[] # a unique list to store all unique class
for l in fp:
  word=""
  tag=""
  for i in l.split("\n"):
    w=i.split(" _ _ ")[0]
    t=i.split(" _ _ ")[1]

    word+=w+" "
    tag+=t+" "

    if t not in d:
      d.append(t)


    # fn.write(s)
  # count+=1
  word=word[:len(word)-1]
  tag=tag[:len(word)-1]
  x.append(word)
  y.append(tag.split())
unique_tag_num=len(d)
print(x[0])
print(y[0])
print(unique_tag_num)

তার মৃত্যুর দশ দিন পর, ১১৫ কৃষ্ণাঙ্গ উচ্চ বিদ্যালয়ের শিক্ষার্থীরা তার হত্যার প্রতিবাদে ম্যাককম্ব এর মাধ্যমে মিছিল করেছে।
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'O', 'O']
13


In [23]:
labels_to_ids = {k: v for v, k in enumerate(d)}
ids_to_labels = {v: k for v, k in enumerate(d)}
print(labels_to_ids)
print(ids_to_labels)

{'O': 0, 'B-LOC': 1, 'B-GRP': 2, 'I-GRP': 3, 'B-PROD': 4, 'B-CW': 5, 'I-CW': 6, 'B-CORP': 7, 'B-PER': 8, 'I-PER': 9, 'I-CORP': 10, 'I-PROD': 11, 'I-LOC': 12}
{0: 'O', 1: 'B-LOC', 2: 'B-GRP', 3: 'I-GRP', 4: 'B-PROD', 5: 'B-CW', 6: 'I-CW', 7: 'B-CORP', 8: 'B-PER', 9: 'I-PER', 10: 'I-CORP', 11: 'I-PROD', 12: 'I-LOC'}


In [24]:
from collections import Counter

# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
    common_tokens = sum(common_tokens.values())
    
    # if there are no common tokens then f1 = 0
    if common_tokens == 0:
        return 0
    
    prec = 1.0 * common_tokens / len(pred_tokens)
    rec = 1.0 * common_tokens / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)


In [25]:
label_all_tokens = False

def align_label(qc, labels):
    
    tokenized_inputs = tokenizer(qc, max_length=128, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
    
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None
    label_ids = []

    #print(word_ids)

    for word_idx in word_ids:
        #print(word_idx)
        if word_idx is None:
          label_ids.append(-100)
        elif word_idx != previous_word_idx:
          try:
            label_ids.append(labels_to_ids[labels[word_idx]])
          except:
            label_ids.append(-100)
        else:
          try:
            label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
          except:
            label_ids.append(-100)
        
        #print(label_ids)
        previous_word_idx = word_idx

    return label_ids


In [26]:
# x = [normalize_text('তিনি বর্তমানে জাপানের সাংবিধানিক গণতান্ত্রিক দল এর সদস্য।'), normalize_text('তিনি যুবক হিসেবে শেফিল্ড বুধবার এফ.সি. যোগদান করেন এবং ১৯৫১ সালে তাদের পেশাগত দিক দিয়ে আত্মপ্রকাশ করেন।')]
# y = ['O O B-GRP I-GRP I-GRP I-GRP O O'.split(), 'O O O B-GRP I-GRP I-GRP O O O O O O O O O O O'.split()]

In [27]:
for i in range(len(x)):
  x[i]=normalize_text(x[i])

In [28]:
from sklearn.model_selection import train_test_split

train,test,train_tags,test_tags=train_test_split(x,y,test_size=0.1,train_size=0.9)

print(len(train))
print(len(test))
print(len(train_tags))
print(len(test_tags))

13770
1530
13770
1530


In [29]:
ins = []
outs = []
for i in range(len(train)):
  ins.append(tokenizer(train[i], max_length=128, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt"))
  outs.append(align_label(train[i], train_tags[i]))

In [30]:
class DataSequence(torch.utils.data.Dataset):

    def __init__(self, i, o):

        #lb = [i.split() for i in a]
        self.texts = i
        self.labels = o

    def __len__(self):

        return len(self.labels)

    def get_batch_data(self, idx):

        return self.texts[idx]

    def get_batch_labels(self, idx):

        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):

        batch_data = self.get_batch_data(idx)
        batch_labels = self.get_batch_labels(idx)

        return batch_data, batch_labels


In [31]:
train_dataset = DataSequence(ins, outs)

In [32]:
def align_word_ids(qc):
  
    tokenized_inputs = tokenizer(qc, max_length=128, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
    #print(qc)
    word_ids = tokenized_inputs.word_ids()
    #print(word_ids)
    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(qc):

    text = tokenizer(qc, max_length=512, padding="max_length", truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")
    
    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(qc)).unsqueeze(0).to(device)
    #print(label_ids)

    logits = model(input_id, mask, None)
    #print(logits[0])
    #print(len(logits[0][0]))
    #logits_clean = logits[0][label_ids != -100]
    #print(logits_clean)
    #print(logits[0][0])

    predictions = logits[0][0].argmax(dim=1).tolist()
    #print(predictions)
    prediction_label = [ids_to_labels[i] for i in predictions]
    #print(prediction_label)
    #print(prediction_label)

    #print(input_id)
    a = tokenizer.convert_ids_to_tokens(input_id[0])
    #print(a)
    '''
    print(prediction_label)
    print(len(prediction_label))
    print(a)
    print(len(a))
    '''

    ans = ''
    ind = True
    for i in range(len(label_ids[0])):
      if (label_ids[0][i] != -100):
        ans = ans + ' ' + prediction_label[i]

    return ans

In [33]:
# Training part 
# no of epoch = 15
# opimizer = Adam
# at every epoch EM and F1 scores were calculated on validation set of BanglaRQA
# at every epoch model was also saved


from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm

# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# move model over to detected device
model.to(device)


# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=2e-5)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

for epoch in range(15):
    # set model to train mode
    model.train()
    model_name = 'banglaBERT_model_weights_epoch_' + str(epoch) + '.pth'
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for train_data, train_label in loop:
        # initialize calculated gradients (from prev step)
        
        

        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = train_data['input_ids'].squeeze(1).to(device)
        attention_mask = train_data['attention_mask'].squeeze(1).to(device)
        labels = train_label.to(device)
      
        outputs = model(input_ids=input_ids, 
                          attention_mask=attention_mask, 
                            labels = labels)
        #print(outputs)
        # extract loss
        loss = outputs[0]
        #print(loss)
        # calculate loss for every parameter that needs grad update 
        loss.backward()
        # update parameters
        optim.step()

        #print(outputs)
        #print(model.config.architectures)
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

    # torch.save({
    #     'epoch': epoch,
    #     'model_state_dict': model.state_dict(),
    #     'optimizer_state_dict': optim.state_dict(),
    #     'loss': loss,
        
    #     }, os.path.join(checkpoint_dir,model_name))
    
    optim.zero_grad()
    model.eval()


    # f1_total = 0.0
    # em_total = 0.0

    # with torch.no_grad():
    #   l = len(context_val)
    #   for i in range(l):
    #       pred = evaluate_one_text(question_val[i] + ' হ্যাঁ না । ' + context_val[i])
    #       pred = pred.replace('#', '')
    #       #if(i<10):
    #         #print(pred)
    #       f1_total = f1_total + compute_f1(pred, answer_val[i])
    #       em_total = em_total + compute_exact_match(pred, answer_val[i])

    #   print("Epoch", epoch)
    #   print('F1: ', f1_total/l)
    #   print('EM: ', em_total/l)  

Epoch 14: 100%|██████████| 1722/1722 [05:32<00:00,  5.18it/s, loss=0.000401]


In [52]:
#testing with test data

fp=open("test.txt", "r").read().split("\n\n") #train data

x=[]

for l in fp:
  word=""

  for i in l.split():


    word+=i+" "

  word=word[:len(word)-1]
  x.append(word)
print(x[0])


নিকটতম বিমানবন্দরটি ( ১৫৮ কিমি ) রাজীব গান্ধী আন্তর্জাতিক বিমানবন্দর ।


In [72]:
#testing with dev data
predicted_file=open("dev_pred_labels.txt","a")

with torch.no_grad():
      l = len(x)
      for i in range(l):
        pred = evaluate_one_text(x[i])
        pred = pred[:-1]
        pred=pred.split()
        # print(pred)
        # print(len(x[i].split()),len(pred))
        if len(x[i].split())>len(pred):
          pred+=(['O']*(len(x[i].split())-len(pred)))
        elif len(pred)>len(x[i].split()):
          pred=pred[0:len(pred)-(len(pred)-len(x[i].split()))]
        
        s2=""


        for z in range(len(pred)):

          s2+=pred[z]+"\n"
        s2+="\n"


        predicted_file.write(s2)

In [34]:
# #testing with dev data

# fp=open("dev.txt", "r").read().split("\n\n") #train data
# total_sentence=len(fp)
# # fn=open("newtrain.txt","a")
# # count=1
# x=[]
# y=[]
# for l in fp:
#   word=""
#   tag=""
#   for i in l.split("\n"):
#     w=i.split(" _ _ ")[0]
#     t=i.split(" _ _ ")[1]

#     word+=w+" "
#     tag+=t+" "
#   word=word[:len(word)-1]
#   tag=tag[:len(word)-1]
#   x.append(word)
#   y.append(tag.split())
# print(x[0])
# print(y[0])


তিনি যুবক হিসেবে শেফিল্ড বুধবার এফ.সি. যোগদান করেন এবং ১৯৫১ সালে তাদের পেশাগত দিক দিয়ে আত্মপ্রকাশ করেন।
['O', 'O', 'O', 'B-GRP', 'I-GRP', 'I-GRP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [35]:
# #testing with dev data
# label_file=open("dev_gt_labels.txt","a")
# predicted_file=open("dev_pred_labels.txt","a")

# with torch.no_grad():
#       l = len(x)
#       for i in range(l):
#         pred = evaluate_one_text(x[i])
#         pred = pred[:-1]
#         pred=pred.split()
#         if len(y[i])>len(pred):
#           pred+=['O']*(len(y[i])-len(pred))
#         elif len(pred)>len(y[i]):
#           pred=pred[0:len(pred)-(len(pred)-len(y[i]))]

#         s1=""
#         s2=""


#         for z in range(len(y[i])):
#           if "\n" in y[i][z] or "\n" in pred[z]:
#             print("seriously!!!!!")
#           s1+=(y[i][z]+"\n")

#           s2+=pred[z]+"\n"
#         s1+="\n"
#         s2+="\n"
#         if len(s1)==len(s2):
          
#           label_file.write(s1)
#           predicted_file.write(s2)

In [36]:
# #testing with splitted train data
# label_file=open("dev_gt_labels.txt","a")
# predicted_file=open("dev_pred_labels.txt","a")

# with torch.no_grad():
#       l = len(test)
#       for i in range(l):

#         pred = evaluate_one_text(test[i])
#         pred = pred[:-1]
#         pred=pred.split()
#         if len(test_tags[i])>len(pred):

#           pred+=['O']*(len(test_tags[i])-len(pred))
#         elif len(pred)>len(test_tags[i]):

#           pred=pred[0:len(pred)-(len(pred)-len(test_tags[i]))]

#         s1=""
#         s2=""

#         for z in range(len(test_tags[i])):
#           if "\n" in test_tags[i][z] or "\n" in pred[z]:
#             print("seriously!!!!!")
#           s1+=(test_tags[i][z]+"\n")

#           s2+=pred[z]+"\n"
#         s1+="\n"
#         s2+="\n"
#         if len(s1)==len(s2):
          
#           label_file.write(s1)
#           predicted_file.write(s2)


In [73]:

label_file=open("dev_gt_labels.txt","r").readlines()
predicted_file=open("dev_pred_labels.txt","r").readlines()
print(len(label_file),len(predicted_file))



165656 164672


In [75]:
!python eval_main.py

{
  "P@LOC": 0.0,
  "R@LOC": 0.0,
  "F1@LOC": 0.0,
  "P@PER": 0.0,
  "R@PER": 0.0,
  "F1@PER": 0.0,
  "P@PRO": 0.0,
  "R@PRO": 0.0,
  "F1@PRO": 0.0,
  "P@GR": 0.0,
  "R@GR": 0.0,
  "F1@GR": 0.0,
  "P@PE": 0.0,
  "R@PE": 0.0,
  "F1@PE": 0.0,
  "P@LO": 0.0,
  "R@LO": 0.0,
  "F1@LO": 0.0,
  "P@COR": 0.0,
  "R@COR": 0.0,
  "F1@COR": 0.0,
  "P@C": 0.0,
  "R@C": 0.0,
  "F1@C": 0.0,
  "P@CW": 0.0,
  "R@CW": 0.0,
  "F1@CW": 0.0,
  "P@CORP": 0.0,
  "R@CORP": 0.0,
  "F1@CORP": 0.0,
  "P@PROD": 0.0,
  "R@PROD": 0.0,
  "F1@PROD": 0.0,
  "P@GRP": 0.0015218902927014864,
  "R@GRP": 0.15153494448073154,
  "F1@GRP": 0.0030135153565865234,
  "Precision": 0.0015218902927014864,
  "Recall": 0.01791644142404819,
  "F1": 0.0028054730910243783,
  "MD@R": 0.34388755888485595,
  "MD@P": 0.02921110979913672,
  "MD@F1": 0.053848153768933094,
  "ALLTRUE": 12949,
  "ALLRECALLED": 4453,
  "ALLPRED": 152442
}
Precision 0.0, Recall 0.02, F1 0.0
